## 8장. 영상 매칭과 추적

### 이미지 매칭 : 이미지의 유사도를 이용하여 비슷한 이미지를 찾아내는 단원
### 영상 특징, 키포인트 :  특징점(특징이 되는 부분) 찾기 주로 모서리 부분

### 이미지 매칭

두 이미지 비교, 짝이 맞는 형태의 객체가 있는지 찾아내는 기술. 즉, 두 이미지 간 유사도를 측정하는 작업이다. 특징을 대표할 수 있는 숫자를 특징 벡터 혹은 특징 디스크립터라고 한다.

#### 평균 해시 매칭

이미지 매칭의 한 기법으로 특정 벡터를 구하기 위해 평균값을 사용한다.

1. 이미지를 가로 세로 비율과 무관하게 특정한 크기로 축소한다.
2. 픽셀 전체의 평균값을 구해 각 픽셀의 값이 평균보다 작으면 0, 크면 1로 바꾼다.
3. 0 또는 1로만 구성된 각 픽셀 값을 1행 1열로 반환한다.

avg_hash.py

In [1]:
import cv2

#영상 읽어서 그레이 스케일로 변환
img = cv2.imread('../img/pistol.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 8x8 크기로 축소 ---①
gray = cv2.resize(gray, (16,16))
# 영상의 평균값 구하기 ---②
avg = gray.mean()
# 평균값을 기준으로 0과 1로 변환 ---③
bin = 1 * (gray > avg)
print(bin)

# 2진수 문자열을 16진수 문자열로 변환 ---④
dhash = []
for row in bin.tolist():
    s = ''.join([str(i) for i in row])
    dhash.append('%02x'%(int(s,2)))
dhash = ''.join(dhash)
print(dhash)

cv2.namedWindow('pistol', cv2.WINDOW_GUI_NORMAL)
cv2.imshow('pistol', img)
cv2.waitKey(0)

[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 0 0 1 1 1 1 1 1]
 [1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1]
 [1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1]
 [1 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1]
 [1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1]
 [1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1]
 [1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1]
 [1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1]
 [1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1]]
ffff8000800080008000813fc1ffc1ffc07fc3ffc7ffc7ff87ff87ff87ffc7ff


-1

0으로 구성된 부분이 권총 모양과 닮았다. 권총은 픽셀 값이 0에 가깝고 배경은 255에 가깝기 때문이다.

그리고 위 권총과 비슷한 유사도를 찾는 방법에 사용되는 거리는 여러가지가 있다. 대표적으로 유클리드 거리와 해밍 거리이다. 이때, 평균 해시 매칭에서는 해밍 거리를 이용한다. 유클리드 거리는 자리수가 높아지면 차이가 크게 벌어지지만 해밍 거리는 길이만 같다면 몇 개의 숫자가 다른지만 고려하기 때문이다.

avg_hash_natching.py

In [ ]:
#이미지 폴더 다운 안받음
import cv2
import numpy as np
import glob

# 영상 읽기 및 표시
img = cv2.imread('../img/pistol.jpg')
cv2.imshow('query', img)

# 비교할 영상들이 있는 경로 ---①
search_dir = '../img/101_ObjectCategories'

# 크기를 맞춰줘야 비교가 가능함
# 이미지를 16x16 크기의 평균 해쉬로 변환 ---②
def img2hash(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (16, 16))
    avg = gray.mean()
    bi = 1 * (gray > avg)
    return bi

# 해밍거리 측정 함수 ---③
def hamming_distance(a, b):
    a = a.reshape(1,-1)
    b = b.reshape(1,-1)
    # 같은 자리의 값이 서로 다른 것들의 합
    distance = (a !=b).sum()
    return distance

# 권총 영상의 해쉬 구하기 ---④
query_hash = img2hash(img)

# 이미지 데이타 셋 디렉토리의 모든 영상 파일 경로 ---⑤
img_path = glob.glob(search_dir+'/**/*.jpg')
for path in img_path:
    # 데이타 셋 영상 한개 읽어서 표시 ---⑥
    img = cv2.imread(path)
    cv2.imshow('searching...', img)
    cv2.waitKey(5)
    # 데이타 셋 영상 한개의 해시  ---⑦
    a_hash = img2hash(img)
    # 해밍 거리 산출 ---⑧
    dst = hamming_distance(query_hash, a_hash)
    if dst/256 < 0.25: # 해밍거리 25% 이내만 출력 ---⑨
        print(path, dst/256)
        cv2.imshow(path, img)
cv2.destroyWindow('searching...')
cv2.waitKey(0)
cv2.destroyAllWindows()

#### 템플릿 매칭

특정 물체에 대한 이미지를 준비하고 그 물체가 포함되어 있을 것이라고 예상할 수 있는 이미지와 비교하여 매칭 되는 위치를 찾는 것이다. 이때 미리 준비한 이미지를 템플릿 이미지라고 한다. 템플릿 이미지는 비교할 이미지보다 크기가 항상 작아야 한다.

- cv2.matchTemplate() : 입력 이미지를 템플릿 이미지에 슬라이딩하면서 주어진 메서드에 따라 매칭을 수행한다.
- cv2.minMaxLoc() : 배열에서 최솟값 혹은 최댓값을 구하는 함수 -> 이를 통해 최선의 매칭값과 매칭점을 구할 수 있다.

template_matching.py

In [2]:

import cv2
import numpy as np

# 입력이미지와 템플릿 이미지 읽기
img = cv2.imread('../img/figures.jpg')
template = cv2.imread('../img/taekwonv1.jpg')
th, tw = template.shape[:2]
cv2.imshow('template', template)

# 3가지 매칭 메서드 순회
# cv2.TM_CCOEFF_NORMED : 상관계수 매칭 정규화
# cv2.TM_CCORR_NORMED : 상관관계 매칭 정규화
# cv2.TM_SQDIFF_NORMED : 제곱 차이 매칭 정규화
methods = ['cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR_NORMED', \
                                     'cv2.TM_SQDIFF_NORMED']
for i, method_name in enumerate(methods):
    img_draw = img.copy()
    method = eval(method_name)
    # 템플릿 매칭   ---①
    res = cv2.matchTemplate(img, template, method)
    # 최솟값, 최댓값과 그 좌표 구하기 ---②
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    print(method_name, min_val, max_val, min_loc, max_loc)

    # TM_SQDIFF의 경우 최솟값이 좋은 매칭, 나머지는 그 반대 ---③
    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
        top_left = min_loc
        match_val = min_val
    else:
        top_left = max_loc
        match_val = max_val
    # 매칭 좌표 구해서 사각형 표시   ---④      
    bottom_right = (top_left[0] + tw, top_left[1] + th)
    cv2.rectangle(img_draw, top_left, bottom_right, (0,0,255),2)
    # 매칭 포인트 표시 ---⑤
    cv2.putText(img_draw, str(match_val), top_left, \
                cv2.FONT_HERSHEY_PLAIN, 2,(0,255,0), 1, cv2.LINE_AA)
    cv2.imshow(method_name, img_draw)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.TM_CCOEFF_NORMED -0.1780252307653427 0.5131933093070984 (42, 0) (208, 43)
cv2.TM_CCORR_NORMED 0.827332615852356 0.9238022565841675 (85, 6) (208, 43)
cv2.TM_SQDIFF_NORMED 0.17028295993804932 0.36860838532447815 (208, 43) (86, 7)


이미지에서 태권브이가 어디에 위치하고 있는지를 잘 찾아내었다. 다만 템플릿 매칭은 크기, 방향, 회전 변환에 잘 작동하지 않고 속도가 느리다는 단점이 있다.

### 이미지 특징점

이미지 특징점이란 이미지에서 특징이 되는 부분을 의미한다. 이미지끼리 서로 매칭이 되는지 확인을 할 때 각 이미지에서의 특징이 되는 부분끼리 비교한다. 즉, 이미지 매칭 시 사용하는 것이 바로 특징점이다. 보통 특징점임 되는 부분은 모서리나 코너이다. 따라서 특징점 검출을 다른 말로 코너 검출이라고도 한다.

#### 해라스 코너 검출

어떤 물체를 볼 때 꼭지점을 유심히 보는 경향이 있다. 즉, 물체를 인식할 때 물체의 코너 부분에 관심을 둔다. 이미지 상의 코너를 잘 찾아낸다면 물체를 보다 쉽게 인식할 수 있다. 해라스 코너 검출은 소벨 미분으로 경곗값을 검출하면서 경곗값의 경사도 변화량을 측정하여 변화량이 수직, 수평, 대각선 방향으로 크게 변화하는 것을 코너로 판단한다.

- cv2.cornerHarris()

corner_harris.py

In [6]:

import cv2
import numpy as np

img = cv2.imread('../img/house.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 해리스 코너 검출 ---①
corner = cv2.cornerHarris(gray, 2, 3, 0.04)
# 변화량 결과의 최대값 10% 이상의 좌표 구하기 ---②
coord = np.where(corner > 0.1* corner.max())
coord = np.stack((coord[1], coord[0]), axis=-1)

# 코너 좌표에 동그리미 그리기 ---③
for x, y in coord:
    cv2.circle(img, (x,y), 5, (0,0,255), 1, cv2.LINE_AA)

# 변화량을 영상으로 표현하기 위해서 0~255로 정규화 ---④
# 안해주면 영상으로 표시가 ㅇㄴ된다.
corner_norm = cv2.normalize(corner, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
# 화면에 출력
corner_norm = cv2.cvtColor(corner_norm, cv2.COLOR_GRAY2BGR)
merged = np.hstack((corner_norm, img))
cv2.imshow('Harris Corner', merged)
cv2.waitKey()
cv2.destroyAllWindows()

코너에서 픽셀의 변화량이 크기 때문에 위 검출을 이용하면 거의 코너 부분임을 알 수 있다.

#### 시-토마시 검출

해리스 코너 검출을 조금 더 개선한 알고리즘

- cv2.goodFeaturesToTrack()

corner_goodFeature.py

In [7]:
import cv2
import numpy as np

img = cv2.imread('../img/house.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 시-토마스의 코너 검출 메서드
corners = cv2.goodFeaturesToTrack(gray, 80, 0.01, 10)
# 실수 좌표를 정수 좌표로 변환
corners = np.int32(corners)

# 좌표에 동그라미 표시
for corner in corners:
    x, y = corner[0]
    cv2.circle(img, (x, y), 5, (0,0,255), 1, cv2.LINE_AA)

cv2.imshow('Corners', img)
cv2.waitKey()
cv2.destroyAllWindows()

앞선 해라스 코너 검출보다 더 많은 양을 검출시킨 것을 확인할 수 있다.

### 특징점 검출기

특징점도 각각의 x,y 좌표를 가진다. 해리스 검출과 시-토마시 검출은 함수 변환 결과 특징점의 좌표를 얻을 수 있다. 지금부터 배울 특징점 검출기들은 특징점의 좌표 뿐 아니라 다양한 정보들도 함께 반환한다.

- detector.detect() : 특징점 검출 함수
- keypoint : 특징점 정보를 담는 객체
- cv2.drawKetpoints()

#### GFTTDetector

cv2.goodFeaturesToTrack() 함수를 이용한 특징점 검출기이다. GFTTDetector 함수의 생성은 밑의 함수와 같다.

- cv2.GFTTFetector_Create()

kpt_gftt.py

In [9]:
import cv2
import numpy as np
 
img = cv2.imread("../img/house.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Good feature to trac 검출기 생성 ---①
gftt = cv2.GFTTDetector_create() 
# 특징점 검출 ---②
keypoints = gftt.detect(gray, None)
# 특징점 그리기 ---③
img_draw = cv2.drawKeypoints(img, keypoints, None) # 색상 지정 안하면 랜덤 색

# 결과 출력 ---④
cv2.imshow('GFTTDectector', img_draw)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### FAST(Feature from Accelerated Segment Test)

기존 검출기보다 속도가 빠르다. FAST 검출기는 코너를 검출할 때 미분 연산을 하지 않고 픽셀을 중심으로 특정 개수의 픽셀로 원을 그려 그 안의 픽셀들의 중심 픽셀값보다 임계값 이상 밝거나 어두운 것이 일정 개수 이상 연속되면 코너로 판단한다.

- cv2.FastFeatureDetector_create()

kpt_fast.py

In [10]:
import cv2
import numpy as np

img = cv2.imread('../img/house.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# FASt 특징 검출기 생성 ---①
fast = cv2.FastFeatureDetector_create(50)
# 특징점 검출 ---②
keypoints = fast.detect(gray, None)
# 특징점 그리기 ---③
img = cv2.drawKeypoints(img, keypoints, None)
# 결과 출력 ---④
cv2.imshow('FAST', img)
cv2.waitKey()
cv2.destroyAllWindows()

#### SimpleBlobDetector

BLOB는 Binary Large OBject의 줄임말로 이진 스케일로 연결된 픽셀 그룹을 의미한다. SimpleBlobDetector는 자잘한 객체는 노이즈로 여기고 특정 크기 이상의 큰 객체만 찾아내는 검출기이다.

- cv2.SimpleBlobDetecor_create() : BLOB 검출기 생성자

kpt_blob.py

In [11]:
import cv2
import numpy as np
 
img = cv2.imread("../img/house.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# SimpleBlobDetector 생성 ---①
detector = cv2.SimpleBlobDetector_create()
# 키 포인트 검출 ---②
keypoints = detector.detect(gray)
# 키 포인트를 빨간색으로 표시 ---③
img = cv2.drawKeypoints(img, keypoints, None, (0,0,255),\
                flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
 
cv2.imshow("Blob", img)
cv2.waitKey(0)

-1

그렇게 썩 많은 코너를 추출하진 못했다. 자잘한 코너는 노이즈로 간주하기 때문이다. 필터 옵션을 바꾸어 실행해보면 다음과 같은 결과를 얻을 수 있다.

kpt_blob_param.py

In [12]:
import cv2
import numpy as np
 
img = cv2.imread("../img/house.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# blob 검출 필터 파라미터 생성 ---①
params = cv2.SimpleBlobDetector_Params()

# 경계값 조정 ---②
params.minThreshold = 10
params.maxThreshold = 240
params.thresholdStep = 5
# 면적 필터 켜고 최소 값 지정 ---③
params.filterByArea = True
params.minArea = 200
  
# 컬러, 볼록 비율, 원형비율 필터 옵션 끄기 ---④
params.filterByColor = False
params.filterByConvexity = False
params.filterByInertia = False
params.filterByCircularity = False 

# 필터 파라미터로 blob 검출기 생성 ---⑤
detector = cv2.SimpleBlobDetector_create(params)
# 키 포인트 검출 ---⑥
keypoints = detector.detect(gray)
# 키 포인트 그리기 ---⑦
img_draw = cv2.drawKeypoints(img, keypoints, None, None,\
                     cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
# 결과 출력 ---⑧
cv2.imshow("Blob with Params", img_draw)
cv2.waitKey(0)

-1

옵션 추가 결과 이전보다는 더 많은 객체가 검출된 것을 확인할 수 있다.

### 특징 디스크립터

특징점은 객체의 좌표 뿐 아니라 주변 픽셀과의 관계에 대한 정보를 가진다. 대표적으로 size, angle 속성, 경사도, 방향성 등이다. 특징 디스크립터란 특징점 주변 픽셀을 일정한 크기의 블록으로 나누어 블록이 속한 픽셀의 그레디언트 히스토그램을 계산한 것이다. 주로, 특징점 주변의 밝기, 색상, 방향, 크기 드의 정보가 포함되어 있다.

- detector.compute() : 특징점을 전달하면 특징 디스크립터를 계산해 반환
- detector.detectAndCompute() : 특징점 검출과 특징 디스트립터 계산을 한 번에 수행한다.

특징 디스크립터를 계산해주는 알고리즘에 대하여 공부해보자.

#### SIFT(Scale-Invariant Feature Transform)

기존의 해리스 코너 검출 알고리즘은 크기 변화에 민감한 문제를 가지고 있었는데 SIFT는 이미지 피라미드(크기를 작은 순부터 큰 순서대로)를 이용해 크기 변화에 따른 특징점 검출 문제를 해결한 알고리즘이다.

- cv2.xfeature2d.SIFT_create()

desc_sift.py

In [13]:
import cv2
import numpy as np

img = cv2.imread('../img/house.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# SIFT 추출기 생성
sift = cv2.xfeatures2d.SIFT_create()
# 키 포인트 검출과 서술자 계산
keypoints, descriptor = sift.detectAndCompute(gray, None)
print('keypoint:',len(keypoints), 'descriptor:', descriptor.shape)
print(descriptor)

# 키 포인트 그리기
img_draw = cv2.drawKeypoints(img, keypoints, None, \
                flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
# 결과 출력
cv2.imshow('SIFT', img_draw)
cv2.waitKey()
cv2.destroyAllWindows()

keypoint: 413 descriptor: (413, 128)
[[  1.   1.   1. ...   0.   0.   1.]
 [  8.  24.   0. ...   1.   0.   4.]
 [  0.   0.   0. ...   0.   0.   2.]
 ...
 [  1.   8.  71. ...  73. 127.   3.]
 [ 35.   2.   7. ...   0.   0.   9.]
 [ 36.  34.   3. ...   0.   0.   1.]]


특징점은 413개이고 특징점 하나 당 128개의 특징 디스크립터 값을 사용하는 것을 알 수 있다.

#### SURF(Speeded Up Robust Features)

SIFT는 이미지 피라미드를 이용하기 때문에 속도가 느릴 수 밖에 없는데 SURF는 이미지 피라미드 대신 필터의 크기를 변화시키는 방식으로 성능을 개선한 알고리즘이다.

- cv2.xfeatures2d.SURF_create()

desc_surf.py

In [19]:
import cv2
import numpy as np

img = cv2.imread('../img/house.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# SURF 추출기 생성 ( 경계:1000, 피라미드:3, 서술자확장:True, 방향적용:True)
surf = cv2.xfeatures2d.SURF_create(1000, 3, True, True)
# 키 포인트 검출 및 서술자 계산
keypoints, desc = surf.detectAndCompute(gray, None)
print(desc.shape, desc)
# 키포인트 이미지에 그리기
img_draw = cv2.drawKeypoints(img, keypoints, None, \
                flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv2.imshow('SURF', img_draw)
cv2.waitKey()
cv2.destroyAllWindows()

TypeError: Argument 'nOctaveLayers' must be integer, not bool

#### ORB(Oriented and Rotated BRIEF)

desc_orb.py

In [ ]:
import cv2
import numpy as np

img = cv2.imread('../img/house.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# ORB 추출기 생성
orb = cv2.ORB_create()
# 키 포인트 검출과 서술자 계산
keypoints, descriptor = orb.detectAndCompute(img, None)
# 키 포인트 그리기
img_draw = cv2.drawKeypoints(img, keypoints, None, \
             flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
# 결과 출력
cv2.imshow('ORB', img_draw)
cv2.waitKey()
cv2.destroyAllWindows()